### 1. `SystemMessage`, `HumanMessage`, `AIMessage`


In [1]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="System message: Instruction to the system"),
        ("system", "System message"),
        HumanMessage(content="Human message: Utterance of the user"),
        ("human", "Human message"),
        ("user", "Human message"),
        AIMessage(content="AI message: Response from the system"),
        ("ai", "AI message"),
        ("assistant", "AI message"),
    ]
)
val = template.invoke({})
print(val.to_string())

System: System message: Instruction to the system
System: System message
Human: Human message: Utterance of the user
Human: Human message
Human: Human message
AI: AI message: Response from the system
AI: AI message
AI: AI message


### 2. `FunctionMessage` with `intermediate_steps`


In [2]:
import json

from langchain.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.agents import AgentActionMessageLog
from langchain.agents.format_scratchpad import format_to_openai_function_messages


def generate_artificial_tool_response(tool: str, tool_input: dict):
    """Generate artificial tool response.

    Args:
        tool (str): The tool.
        tool_input (dict): The tool input.

    Returns:
        dict: The artificial tool response.
    """
    return AgentActionMessageLog(
        tool,
        tool_input,
        log=f"\nInvoking: `{tool}` with `{tool_input}`\n\n\n",
        message_log=[
            AIMessage(
                content="",
                additional_kwargs={
                    "function_call": {
                        "arguments": json.dumps(tool_input, ensure_ascii=False),
                        "name": tool,
                    }
                },
            )
        ],
    )


agent_action = generate_artificial_tool_response(
    tool="request_again",
    tool_input={"utterance": "지나친 세일은 게임의 가치를 낮출 뿐입니다"},
)
observation = "75% 세일을 하지 않을 것입니다"

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are function calling assistant"),
        HumanMessage(content="있잖아 그거해봐 그거"),
        AIMessage(content="어.. 또..?"),
        HumanMessage(content="해봐! 빨리!"),
        *format_to_openai_function_messages([(agent_action, observation)]),
    ]
)
val = template.invoke({})
print(val.to_string())

System: You are function calling assistant
Human: 있잖아 그거해봐 그거
AI: 어.. 또..?
Human: 해봐! 빨리!
AI: {'arguments': '{"utterance": "지나친 세일은 게임의 가치를 낮출 뿐입니다"}', 'name': 'request_again'}
Function: 75% 세일을 하지 않을 것입니다


In [3]:
import json

from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.agents import AgentActionMessageLog
from langchain.agents.format_scratchpad import format_to_openai_function_messages


def generate_artificial_tool_response(tool: str, tool_input: dict):
    """Generate artificial tool response.

    Args:
        tool (str): The tool.
        tool_input (dict): The tool input.

    Returns:
        dict: The artificial tool response.
    """
    return AgentActionMessageLog(
        tool,
        tool_input,
        log=f"\nInvoking: `{tool}` with `{tool_input}`\n\n\n",
        message_log=[
            AIMessage(
                content="",
                additional_kwargs={
                    "function_call": {
                        "arguments": json.dumps(tool_input, ensure_ascii=False),
                        "name": tool,
                    }
                },
            )
        ],
    )


agent_action = generate_artificial_tool_response(
    tool="request_again",
    tool_input={"utterance": "지나친 세일은 게임의 가치를 낮출 뿐입니다"},
)
observation = "75% 세일을 하지 않을 것입니다"

preprocess = RunnablePassthrough.assign(
    agent_scratchpad=lambda input: format_to_openai_function_messages(
        input["intermediate_steps"]
    )
)
template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are function calling assistant"),
        HumanMessage(content="있잖아 그거해봐 그거"),
        AIMessage(content="어.. 또..?"),
        HumanMessage(content="해봐! 빨리!"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
template_with_preprocess = preprocess | template
val = template_with_preprocess.invoke(
    {"intermediate_steps": [(agent_action, observation)]}
)
print(val.to_string())

System: You are function calling assistant
Human: 있잖아 그거해봐 그거
AI: 어.. 또..?
Human: 해봐! 빨리!
AI: {'arguments': '{"utterance": "지나친 세일은 게임의 가치를 낮출 뿐입니다"}', 'name': 'request_again'}
Function: 75% 세일을 하지 않을 것입니다
